In [37]:
!pip install selenium

Defaulting to user installation because normal site-packages is not writeable


In [48]:
from time import sleep, time_ns

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException

import pandas as pd


In [29]:
"""
The code below for chrome browser in linux.
path = "/usr/bin/chromedriver"
option = webdriver.ChromeOptions()
browser = webdriver.Chrome(service=path, options=option)
"""
url = "http://115.124.110.196:8080/epeek/"
browser = webdriver.Firefox()
browser.get(url)
browser.implicitly_wait(2) #wait for two second to load the website fully

#Select the value from the season dropdown
season_drop = Select(browser.find_element(By.ID, 'seasonOpt'))
season_drop.select_by_value('उन्हाळी')
sleep(1)

#Select the value from the division dropdown
div_drop = Select(browser.find_element(By.ID, 'divOpt'))
div_drop.select_by_value('amravati')
sleep(1)

#Select the value from the district dropdown
dist_drop = Select(browser.find_element(By.ID, 'distOpt'))
dist_drop.select_by_value('-')
sleep(1)

#Select the values from the village dropdown
village_drop = Select(browser.find_element(By.ID, 'villOpt'))
village_drop.select_by_value('-')

#Select button
button = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[2]/table/tbody/tr[11]/td/button')
button.click()
sleep(20)

object_content = browser.find_element(By.TAG_NAME, 'object')
browser.switch_to.frame(object_content)

rows = len(browser.find_elements(By.CSS_SELECTOR, 'tr'))
print(rows)
row = 2
data_list = []
while(row<rows-1):
    data = {}
    try:
        data['sl_no'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[1]').text
        data['village'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[2]').text
        data['survey_account_number'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[3]').text
        data['peak_monitoring_area'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[4]').text
        data['permanent_area'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[5]').text
        data['current_area'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[6]').text
        data['total_area'] = browser.find_element(By.XPATH,f'//*[@id="tbl_exporttable_to_xls"]/tbody/tr[{row}]/td[7]').text
        
        data_list.append(data)
        row += 1

    except NoSuchElementException:
        break

print(data_list)

data_set = pd.DataFrame(data_list)
data_set.to_csv(f'data/data_{time_ns()}.csv')


browser.close()



8
[{'sl_no': '1', 'village': 'अकोला', 'survey_account_number': '4069', 'peak_monitoring_area': '5956.97', 'permanent_area': '0.00', 'current_area': '29.76', 'total_area': '5986.73'}, {'sl_no': '2', 'village': 'अमरावती', 'survey_account_number': '3616', 'peak_monitoring_area': '5386.93', 'permanent_area': '0.00', 'current_area': '14.59', 'total_area': '5401.52'}, {'sl_no': '3', 'village': 'बुलडाणा', 'survey_account_number': '9050', 'peak_monitoring_area': '12503.17', 'permanent_area': '0.00', 'current_area': '35.94', 'total_area': '12539.11'}, {'sl_no': '4', 'village': 'यवतमाळ', 'survey_account_number': '3968', 'peak_monitoring_area': '6788.61', 'permanent_area': '0.00', 'current_area': '47.67', 'total_area': '6836.28'}, {'sl_no': '5', 'village': 'वाशिम', 'survey_account_number': '1364', 'peak_monitoring_area': '2125.23', 'permanent_area': '0.00', 'current_area': '16.05', 'total_area': '2141.29'}]


In [3]:
!pip install beautifulsoup4

Defaulting to user installation because normal site-packages is not writeable


In [5]:
#Imports required packages
import requests
from bs4 import BeautifulSoup as bs

In [15]:
URL = "http://115.124.110.196:8080/epeek/rptViewDist.jsp?ccode=-&blk=-&dist=-&div=amravati&season=खरीप&divName=अमरावती&distName=सर्व&blkName=सर्व&ccodeName=सर्व&seasonName=खरीप"
page = requests.get(URL)
print(page)
# html_content = bs(page.content, "html.parser")


<Response [200]>


In [18]:
html_content = bs(page.content, "html.parser")

In [21]:
html_content


<script src="js/exportExcel.js" type="text/javascript"></script>
<style>
    td{
        padding: 5px 20px 5px 20px;
    }
    tr:nth-child(odd)
    {
        background: #e6fdf7;
    }
    
    tr:nth-child(even)
    {
        background: #fff;
    }
</style>
<table border="1" id="tbl_exporttable_to_xls" style="border-collapse:collapse; color: #000;margin: 0 auto; margin-top:50px; font-weight:bold; font-family: helvetica;">
<tr style="text-align: center;background-color: #b1d7b7;">
<td style="padding: 5px;">अ.क्र.</td>
<td>जिल्हा</td>
<td>पीक पाहणी खातेदार संख्या</td>
<td>पीक पाहणी क्षेत्र (हे.)</td>
<td>कायम पड क्षेत्र (हे.)</td>
<td>चालू पड क्षेत्र (हे.)</td>
<td>एकूण क्षेत्र (हे.)</td>
</tr>
<tr>
<td style="text-align: right;">1</td>
<td>अकोला</td>
<td style="text-align: right;">126520</td>
<td style="text-align: right;">193468.64</td>
<td style="text-align: right;">0.00</td>
<td style="text-align: right;">1712.91</td>
<td style="text-align: right;">195181.55</td>
</tr>
<tr>
<td s

In [46]:
html_result_table =  html_content.find('table')
rows = html_result_table.find_all('tr')
print(rows[0].find_all('td')[0].text)
print(len(rows))

row = 1
data_list = []
while(row < len(rows)-2):
    data = {}
    data['sl_no'] = rows[row].find_all('td')[0].text.strip()
    data['village'] = rows[row].find_all('td')[1].text.strip()
    data['survey_account_number'] = rows[row].find_all('td')[2].text.strip()
    data['peak_monitoring_area'] = rows[row].find_all('td')[3].text.strip()
    data['permanent_area'] = rows[row].find_all('td')[4].text.strip()
    data['current_area'] = rows[row].find_all('td')[5].text.strip()
    data['total_area'] = rows[row].find_all('td')[6].text.strip()
    row +=1
    data_list.append(data)

print(data_list)

अ.क्र.
8
[{'sl_no': '1', 'village': 'अकोला', 'survey_account_number': '126520', 'peak_monitoring_area': '193468.64', 'permanent_area': '0.00', 'current_area': '1712.91', 'total_area': '195181.55'}, {'sl_no': '2', 'village': 'अमरावती', 'survey_account_number': '228205', 'peak_monitoring_area': '350231.63', 'permanent_area': '0.00', 'current_area': '4609.60', 'total_area': '354841.22'}, {'sl_no': '3', 'village': 'बुलडाणा', 'survey_account_number': '232168', 'peak_monitoring_area': '331927.81', 'permanent_area': '0.00', 'current_area': '2623.36', 'total_area': '334551.16'}, {'sl_no': '4', 'village': 'यवतमाळ', 'survey_account_number': '225942', 'peak_monitoring_area': '409134.28', 'permanent_area': '0.00', 'current_area': '6432.11', 'total_area': '415566.38'}, {'sl_no': '5', 'village': 'वाशिम', 'survey_account_number': '134089', 'peak_monitoring_area': '216559.00', 'permanent_area': '0.00', 'current_area': '1834.24', 'total_area': '218393.23'}]


In [49]:
data_set = pd.DataFrame(data_list)
data_set

,sl_no,village,survey_account_number,peak_monitoring_area,permanent_area,current_area,total_area
0,1,अकोला,126520,193468.64,0.00,1712.91,195181.55
1,2,अमरावती,228205,350231.63,0.00,4609.60,354841.22
2,3,बुलडाणा,232168,331927.81,0.00,2623.36,334551.16
3,4,यवतमाळ,225942,409134.28,0.00,6432.11,415566.38
4,5,वाशिम,134089,216559.00,0.00,1834.24,218393.23
